In [70]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load the training data into feature matrix, class labels, and event ids:

In [132]:
# predictions 1 is "s", -1 is "b"
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [134]:
print(tX.shape)
tX

(250000, 30)


array([[ 138.47 ,   51.655,   97.827, ...,    1.24 ,   -2.475,  113.497],
       [ 160.937,   68.768,  103.235, ..., -999.   , -999.   ,   46.226],
       [-999.   ,  162.172,  125.953, ..., -999.   , -999.   ,   44.251],
       ...,
       [ 105.457,   60.526,   75.839, ..., -999.   , -999.   ,   41.992],
       [  94.951,   19.362,   68.812, ..., -999.   , -999.   ,    0.   ],
       [-999.   ,   72.756,   70.831, ..., -999.   , -999.   ,    0.   ]])

## Preprocessing

In [135]:
from clean_data import *

# save ind_delete to delete indices from test data
tX_clean, ind_delete = update_X(tX, bound_delete=0.9, bound_change=0.05)

Row 0, freq 0.152456
Row 1, freq 0.0
Row 2, freq 0.0
Row 3, freq 0.0
Row 4, freq 0.709828
Row 5, freq 0.709828
Row 6, freq 0.709828
Row 7, freq 0.0
Row 8, freq 0.0
Row 9, freq 0.0
Row 10, freq 0.0
Row 11, freq 0.0
Row 12, freq 0.709828
Row 13, freq 0.0
Row 14, freq 0.0
Row 15, freq 0.0
Row 16, freq 0.0
Row 17, freq 0.0
Row 18, freq 0.0
Row 19, freq 0.0
Row 20, freq 0.0
Row 21, freq 0.0
Row 22, freq 0.0
Row 23, freq 0.399652
Row 24, freq 0.399652
Row 25, freq 0.399652
Row 26, freq 0.709828
Row 27, freq 0.709828
Row 28, freq 0.709828
Row 29, freq 0.0
112.406
2.107
225.885
-0.244
65.561
47.902


In [224]:
from split_expand_data import *

# split data into train and test sets
X_train, y_train, X_test, y_test = split_data(tX_clean, y, split_ratio=0.7)

# store d to later use with real test set
d=8

# create expanded X_train and X_test and normalize
X_train_poly, mu_train_poly, std_train_poly = expand_and_normalize_X(X_train,d)
X_test_poly  = expand_X(X_test,d)
X_test_poly[:,1:]  = (X_test_poly[:,1:]-mu_train_poly)/std_train_poly

# Do your thing crazy machine learning thing here :) ...

### I - Least Squares GD:

In [27]:
from least_squares_GD import *

initial_w = np.random.normal(0, 1e-1, X_train_poly.shape[1])
max_iters = 300
gamma = 0.008

w, loss = least_squares_GD(y_train, X_train_poly, initial_w, max_iters, gamma)
print(loss)

0.41122017806504824


### II Least Squares SGD

In [28]:
# I think this is shitty cause we have 72 params so needs param optimization

from least_squares_SGD import *

initial_w = np.random.normal(0, 1e-1, X_train_poly.shape[1])
max_iters = 100
gamma = 0.001

w, loss = least_squares_SGD(y_train, X_train_poly, initial_w, max_iters, gamma)
print(loss)

0.9715214812097751


### III Least Squares

In [225]:
from least_squares import *

w, loss_analytical = least_squares(y_train, X_train_poly)

print(loss_analytical)

print(compute_loss_mse(y_test, X_test_poly, w))

0.28039029339137816
0.4324021634665055


### IV Ridge Regression

In [178]:
def ridge_regression(y, tx, lambda_):
    """implement ridge regression."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # ridge regression: TODO
    # ***************************************************
    N = tx.shape[0]
    lambda_prim = 2 * N * lambda_
    w = np.linalg.solve(tx.T@tx + lambda_prim * np.eye(tx.shape[1]), tx.T@y)
    loss = compute_loss_mse(y, tx, w)
    return w, loss

In [56]:
def ridge_regression_demo(x, y, degree, ratio, seed):
    """ridge regression demo."""
    # define parameter
    lambdas = np.logspace(-5, 0, 15)
    # ***************************************************
    # INSERT YOUR CODE HERE
    # split the data, and return train and test data: TODO
    # ***************************************************
    x_train, y_train, x_test, y_test = split_data(x.reshape(-1,1), y, ratio, seed)
    # ***************************************************
    # INSERT YOUR CODE HERE
    # form train and test data with polynomial basis function: TODO
    # ***************************************************
    x_train_poly = expand_X(x_train.reshape(-1,1), degree)
    x_test_poly = expand_X(x_test.reshape(-1,1), degree)

    rmse_tr = []
    rmse_te = []
    for ind, lambda_ in enumerate(lambdas):
        # ***************************************************
        # INSERT YOUR CODE HERE
        # ridge regression with a given lambda
        # ***************************************************
        w = ridge_regression(y_train, x_train_poly, lambda_)[0]
        rmse_tr.append(np.sqrt(2 * compute_loss_mse(y_train, x_train_poly, w)))
        rmse_te.append(np.sqrt(2 * compute_loss_mse(y_test, x_test_poly, w)))
        print("proportion={p}, degree={d}, lambda={l:.3f}, Training RMSE={tr:.3f}, Testing RMSE={te:.3f}".format(
               p=ratio, d=degree, l=lambda_, tr=rmse_tr[ind], te=rmse_te[ind]))
        
    # Plot the obtained results
    plot_train_test(rmse_tr, rmse_te, lambdas, degree)

In [65]:
def plot_train_test(train_errors, test_errors, lambdas, degree):
    """
    train_errors, test_errors and lambas should be list (of the same size) the respective train error and test error for a given lambda,
    * lambda[0] = 1
    * train_errors[0] = RMSE of a ridge regression on the train set
    * test_errors[0] = RMSE of the parameter found by ridge regression applied on the test set
    
    degree is just used for the title of the plot.
    """
    plt.semilogx(lambdas, train_errors, color='b', marker='*', label="Train error")
    plt.semilogx(lambdas, test_errors, color='r', marker='*', label="Test error")
    plt.xlabel("lambda")
    plt.ylabel("RMSE")
    plt.title("Ridge regression for polynomial degree " + str(degree))
    leg = plt.legend(loc=1, shadow=True)
    leg.draw_frame(False)
    plt.savefig("ridge_regression")

In [228]:
lambdas = np.logspace(-4, -1, 20)

ws = []
rmse_tr = []
rmse_te = []

for ind, lambda_ in enumerate(lambdas):
        # ***************************************************
        # INSERT YOUR CODE HERE
        # ridge regression with a given lambda
        # ***************************************************
        ws.append(ridge_regression(y_train, X_train_poly, lambda_)[0])
        rmse_tr.append(compute_loss_mse(y_train, X_train_poly, ws[ind]))
        rmse_te.append(compute_loss_mse(y_test, X_test_poly, ws[ind]))
        print("lambda={l:.3f}, Training RMSE={tr:.3f}, Testing RMSE={te:.3f}".format(
               l=lambda_, tr=rmse_tr[ind], te=rmse_te[ind]))

lambda=0.000, Training RMSE=0.297, Testing RMSE=0.298
lambda=0.000, Training RMSE=0.298, Testing RMSE=0.298
lambda=0.000, Training RMSE=0.298, Testing RMSE=0.299
lambda=0.000, Training RMSE=0.299, Testing RMSE=0.299
lambda=0.000, Training RMSE=0.299, Testing RMSE=0.300
lambda=0.001, Training RMSE=0.300, Testing RMSE=0.301
lambda=0.001, Training RMSE=0.301, Testing RMSE=0.302
lambda=0.001, Training RMSE=0.302, Testing RMSE=0.303
lambda=0.002, Training RMSE=0.303, Testing RMSE=0.305
lambda=0.003, Training RMSE=0.305, Testing RMSE=0.307
lambda=0.004, Training RMSE=0.306, Testing RMSE=0.308
lambda=0.005, Training RMSE=0.307, Testing RMSE=0.309
lambda=0.008, Training RMSE=0.309, Testing RMSE=0.310
lambda=0.011, Training RMSE=0.311, Testing RMSE=0.312
lambda=0.016, Training RMSE=0.313, Testing RMSE=0.313
lambda=0.023, Training RMSE=0.315, Testing RMSE=0.315
lambda=0.034, Training RMSE=0.318, Testing RMSE=0.318
lambda=0.048, Training RMSE=0.321, Testing RMSE=0.321
lambda=0.070, Training RMSE=

### V Logistic Regression

In [230]:
from logistic_regression import *

y_train_log = y_train
y_test_log = y_test

y_train_log[y_train == -1] = 0
y_test_log[y_test == -1] = 0


max_iters = 80
initial_w = np.random.normal(0, 1e-4, X_train_poly.shape[1])
lambdas = np.logspace(-7, -4, 5)

# for lambda_ in lambdas
ws = []
losses = []

for ind, lambda_ in enumerate(lambdas):
    w, loss = logistic_regression(y_train_log, X_train_poly, initial_w, max_iters, lambda_)
    ws.append(w)
    losses.append(loss)
    print("lambda={l:.10f}, Loss sigmoid={loss:.3f}".format(
               l=lambda_, loss=losses[ind]))

print(loss)

lambda=0.0000001000, Loss sigmoid=98980.363
lambda=0.0000005623, Loss sigmoid=88212.407
lambda=0.0000031623, Loss sigmoid=81957.533
lambda=0.0000177828, Loss sigmoid=inf
lambda=0.0001000000, Loss sigmoid=inf
inf


### VI Regularized Logistic Regression

### A)* Cross Validation (do we need it???)

In [11]:
def fold_indices(num_examples,k_fold):
    """
    Splits data indices
    num_examples: total samples in the dataset
    k_fold: number fold of Cross Validation
    
    Returns
    -------
    array of shuffled indices with shape (k_fold, num_examples//k_fold)
    """
    ind = np.arange(num_examples)
    split_size = num_examples//k_fold
    
    # shuffle data
    np.random.shuffle(ind)
    
    k_fold_indices = []
    # Generate k_fold set of indices
    k_fold_indices = [ind[k*split_size:(k+1)*split_size] for k in range(k_fold)]
         
    return np.array(k_fold_indices)

## Predictions based on w

In [226]:
# from helpers

p = predict_labels(w, X_test_poly)

In [227]:
print('Train Accuracy: %.1f %%' % (np.mean(p == y_test) * 100))

Train Accuracy: 81.1 %


In [ ]:
# predict log

In [231]:
# check for lambdas

for i, w in enumerate(ws):
    p = np.where(X_test_poly@w > 0.5, 1, 0)
    print("Lambda: {}".format(lambdas[i]))
    print('Train Accuracy: %.1f %%' % (np.mean(p == y_test_log) * 100))

Lambda: 1e-07
Train Accuracy: 72.3 %
Lambda: 5.62341325190349e-07
Train Accuracy: 73.4 %
Lambda: 3.162277660168379e-06
Train Accuracy: 75.3 %
Lambda: 1.778279410038923e-05
Train Accuracy: 70.9 %
Lambda: 0.0001
Train Accuracy: 72.1 %


In [229]:
# check for lambdas

for i, w in enumerate(ws):
    p = predict_labels(w, X_test_poly)
    print("Lambda: {}".format(lambdas[i]))
    print('Train Accuracy: %.1f %%' % (np.mean(p == y_test) * 100))

Lambda: 0.0001
Train Accuracy: 79.4 %
Lambda: 0.0001438449888287663
Train Accuracy: 79.4 %
Lambda: 0.00020691380811147902
Train Accuracy: 79.3 %
Lambda: 0.00029763514416313193
Train Accuracy: 79.2 %
Lambda: 0.00042813323987193956
Train Accuracy: 79.1 %
Lambda: 0.0006158482110660267
Train Accuracy: 79.0 %
Lambda: 0.0008858667904100823
Train Accuracy: 78.9 %
Lambda: 0.0012742749857031334
Train Accuracy: 78.8 %
Lambda: 0.0018329807108324356
Train Accuracy: 78.6 %
Lambda: 0.0026366508987303583
Train Accuracy: 78.5 %
Lambda: 0.00379269019073225
Train Accuracy: 78.3 %
Lambda: 0.005455594781168515
Train Accuracy: 78.1 %
Lambda: 0.007847599703514606
Train Accuracy: 77.9 %
Lambda: 0.011288378916846883
Train Accuracy: 77.8 %
Lambda: 0.01623776739188721
Train Accuracy: 77.6 %
Lambda: 0.023357214690901212
Train Accuracy: 77.3 %
Lambda: 0.03359818286283781
Train Accuracy: 77.1 %
Lambda: 0.04832930238571752
Train Accuracy: 76.8 %
Lambda: 0.06951927961775606
Train Accuracy: 76.5 %
Lambda: 0.1
Train A

## Obtaining results

In [143]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download test data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)


In [144]:
# tX_test = np.delete(tX_test, ind_delete, axis=1)
tX_test = update_outliers(tX_test, [0, 4, 5, 6, 23, 26], [12, 24, 25, 27, 28])

tX_test_poly  = expand_X(tX_test,d)
tX_test_poly[:,1:]  = (tX_test_poly[:,1:]-mu_train_poly)/std_train_poly

112.541
2.099
226.345
-0.244
65.839
48.037


In [148]:
OUTPUT_PATH = '../data/submission.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(w, tX_test_poly)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)